In [1]:
val x = 2
val y = 3
x + y

Intitializing Scala interpreter ...

Spark Web UI available at http://deborahs-air:4040
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1562354452959)
SparkSession available as 'spark'


x: Int = 2
y: Int = 3
res0: Int = 5


In [4]:
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.log4j._
import org.apache.spark.sql.SparkSession
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.MatrixFactorizationModel
import org.apache.spark.mllib.recommendation.Rating
import scala.Tuple2
import org.apache.spark.rdd.RDD

import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.log4j._
import org.apache.spark.sql.SparkSession
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.MatrixFactorizationModel
import org.apache.spark.mllib.recommendation.Rating
import scala.Tuple2
import org.apache.spark.rdd.RDD


In [5]:
Logger.getLogger("org").setLevel(Level.ERROR)

In [ ]:
final case class Movie(movieID: Int, userID: Int, rating: int)

In [42]:
val spark = SparkSession
    .builder
    .appName("MovieLens Recommendation")
    .master("local[*]")
    .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@35f99d89


In [43]:
//val ratingsFile = spark.sparkContext.textFile("/Users/deborahgemellaro/Programming/612/project_5/ml-latest-small/ratings.csv")
val ratingsFile = "/Users/deborahgemellaro/Programming/612/project_5/ml-latest-small/ratings.csv"
val moviesFile = "/Users/deborahgemellaro/Programming/612/project_5/ml-latest-small/movies.csv"


ratingsFile: String = /Users/deborahgemellaro/Programming/612/project_5/ml-latest-small/ratings.csv
moviesFile: String = /Users/deborahgemellaro/Programming/612/project_5/ml-latest-small/movies.csv


In [44]:
// Convert to a dataframe
import spark.implicits._


<console>: 65: error: value toDF is not a member of String

In [46]:
val df1 = spark.read.format("com.databricks.spark.csv").option("header", true).load(ratingsFile)
df1.printSchema

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



df1: org.apache.spark.sql.DataFrame = [userId: string, movieId: string ... 2 more fields]


In [47]:
val ratingsDF = df1.select(df1.col("userId"), 
                           df1.col("movieId"), 
                           df1.col("rating"), 
                           df1.col("timestamp"))
ratingsDF.show()


+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



ratingsDF: org.apache.spark.sql.DataFrame = [userId: string, movieId: string ... 2 more fields]


In [48]:
val df2 = spark.read.format("com.databricks.spark.csv").option("header", true).load(moviesFile)
df2.printSchema

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



df2: org.apache.spark.sql.DataFrame = [movieId: string, title: string ... 1 more field]


In [49]:
val moviesDF = df2.select(df2.col("movieId"), 
                          df2.col("title"),
                          df2.col("genres"))
moviesDF.show

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

moviesDF: org.apache.spark.sql.DataFrame = [movieId: string, title: string ... 1 more field]


In [55]:
// Register both DataFrames as temp tables to make querying easier
ratingsDF.createOrReplaceTempView("ratings")
moviesDF.createOrReplaceTempView("movies")

In [51]:
// Count total number of ratings
ratingsDF.count()

res29: Long = 100836


In [52]:
// Count number of distinct users
ratingsDF.select(ratingsDF.col("userId")).distinct().count()

res30: Long = 610


In [53]:
// Count number of distinct movies
ratingsDF.select(ratingsDF.col("movieId")).distinct().count()

res31: Long = 9724


In [87]:
val results - spark.sql("SELECT movies.title, movierates.maxr, movierates.minr, movierates.cntu FROM(SELECT ratings.movieId, MAX(ratings.rating) as maxr, MIN(ratings.rating) as minr, COUNT(DISTINCT userId) as cntu FROM ratings GROUP BY ratings.movieId) movierates JOIN movies on movierates.movieId=movies.movieId ORDER BY movierates.cntu DESC")

<console>:  error: incomplete input

In [85]:
val newTable = spark.sql("SELECT ratings.userId, ratings.movieID, ratings.rating, movies.title FROM ratings JOIN movies ON movies.movieId=ratings.movieId WHERE ratings.userId=345 and ratings.rating > 4")
                        
newTable.show()

+------+-------+------+--------------------+
|userId|movieID|rating|               title|
+------+-------+------+--------------------+
|   345|    779|   5.0|'Til There Was Yo...|
|   345|    838|   5.0|         Emma (1996)|
|   345|    926|   5.0|All About Eve (1950)|
|   345|    932|   5.0|Affair to Remembe...|
|   345|   1203|   5.0| 12 Angry Men (1957)|
|   345|   1214|   5.0|        Alien (1979)|
|   345|   2359|   5.0|Waking Ned Devine...|
|   345|   2971|   5.0|All That Jazz (1979)|
|   345|   7121|   5.0|   Adam's Rib (1949)|
|   345|  27751|   5.0| 'Salem's Lot (2004)|
+------+-------+------+--------------------+



newTable: org.apache.spark.sql.DataFrame = [userId: string, movieID: string ... 2 more fields]


In [88]:
val splits = ratingsDF.randomSplit(Array(0.75, 0.25), seed = 123L)
val (trainingData, testData) = (splits(0), splits(1))
val numTraining = trainingData.count()
val numTest = testData.count()
println("Training: " + numTraining + " Test: " + numTest)

Training: 75834 Test: 25002


splits: Array[org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]] = Array([userId: string, movieId: string ... 2 more fields], [userId: string, movieId: string ... 2 more fields])
trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: string, movieId: string ... 2 more fields]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [userId: string, movieId: string ... 2 more fields]
numTraining: Long = 75834
numTest: Long = 25002


In [91]:
val ratingsRDD = trainingData.rdd.map(row => {
    val userId = row.getString(0)
    val movieId = row.getString(1)
    val ratings = row.getString(2)
    Rating(userId.toInt, movieId.toInt, ratings.toDouble)
})

ratingsRDD: org.apache.spark.rdd.RDD[org.apache.spark.mllib.recommendation.Rating] = MapPartitionsRDD[256] at map at <console>:61


In [ ]:
// ALS
